# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（関数名・引数名・返却列・順序）
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `各ノードを Root / Inner / Leaf に分類して返す。`
* 入力 DF: `Tree(id: int, p_id: int | NaN)`
* 出力: `id, type`

  * `p_id` が `NaN` → `"Root"`
  * 親あり かつ 子なし → `"Leaf"`
  * 親あり かつ 子あり → `"Inner"`

## 2) 実装（指定シグネチャ厳守）

> 列最小化 → 親ID集合（`p_id`）のユニーク化 → `isin` と `np.where` のベクトル化判定。
> `groupby` や `merge` は不要で、**O(N)**・1パスで分類します。

```python
import pandas as pd
import numpy as np

def classify_tree_nodes(tree: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        tree (pd.DataFrame): 列 'id', 'p_id' を持つデータフレーム
    Returns:
        pd.DataFrame: 列名と順序は ['id', 'type']
    """
    # 列最小化
    t = tree[['id', 'p_id']].copy()

    # 子を持つノード集合（= p_id に出現する id）
    parent_ids = pd.Index(t['p_id'].dropna().unique())

    # ベクトル化判定
    is_root = t['p_id'].isna()
    has_child = t['id'].isin(parent_ids)

    node_type = np.where(
        is_root, 'Root',
        np.where(has_child, 'Inner', 'Leaf')
    )

    # 出力（列順固定）
    out = pd.DataFrame({'id': t['id'].values, 'type': node_type})
    return out

Analyze Complexity
Runtime 282 ms
Beats 86.82%
Memory 67.08 MB
Beats 64.44%

```

## 3) アルゴリズム説明

* 使用 API

  * `Series.dropna().unique()`：`p_id` のユニーク親集合を抽出
  * `Index.isin`：各 `id` が親集合に含まれるかを高速判定
  * `np.where`：三分岐（Root / Inner / Leaf）をベクトル化で計算
* **NULL / 重複 / 型**

  * ルート判定は `p_id.isna()` を明示
  * `p_id` に `NaN` があるため、親集合作成時は `dropna()` を必ず通す
  * 型はそのまま扱い、`id` は入力のまま返却（順序指定が無いので並べ替えなし）

## 4) 計算量（概算）

* 親集合の作成：`dropna` + `unique` → **O(N)**
* `isin` による包含判定：**O(N)**
* 全体：**O(N)**、追加メモリは `parent_ids` と `node_type` 分

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 データフレーム Tree]
  B[列最小化 id p_id のみ]
  C[p_id から親ID集合を作成 unique]
  D[id が親集合に含まれるかを判定]
  E[p_id が NaN なら Root そうで親ありは Inner それ以外 Leaf]
  F[出力 id type]
  A --> B
  B --> C
  B --> D
  C --> D
  D --> E
  E --> F
```

さらに数％〜2桁％の改善が狙える**配列直叩き（NumPy 主体）**版をご提案します。ポイントは「**DataFrame/Index を極力作らず**、`Series.values/to_numpy()` と `np.in1d`・ブール代入で一気に仕上げる」ことです。

# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**
* I/O 禁止、`print` / `sort_values` 不要

## 1) 問題

* `各ノードを Root / Inner / Leaf に分類して返す。`
* 入力 DF: `Tree(id: int, p_id: int | NaN)`
* 出力: `id, type`（順序任意）

## 2) 実装（指定シグネチャ厳守）

> **DataFrame/Index オブジェクト化を避ける**→ `Series.to_numpy()` と `np.in1d(assume_unique=True)` を活用。
> `id` は一意という仕様を前提に `assume_unique=True` を使い、`np.where` ではなく**ブール代入**で 3 区分を埋めます。

```python
import pandas as pd
import numpy as np

def classify_tree_nodes(tree: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        tree (pd.DataFrame): 列 'id', 'p_id' を持つデータフレーム
    Returns:
        pd.DataFrame: 列名と順序は ['id', 'type']
    """
    # 参照だけ取得（コピーを作らない）
    id_s = tree['id']
    p_s  = tree['p_id']

    # 配列として取得（na_value の指定は不要。pandas は NaN で来る）
    ids = id_s.to_numpy()
    parents = p_s.to_numpy()

    # 親IDのユニーク集合（NaN 除去）を「配列」で確保
    mask_parent = ~pd.isna(parents)
    parent_unique = pd.unique(parents[mask_parent])

    # 子を持つか（id ∈ parent_unique）
    # id は一意という仕様 → assume_unique=True が安全に使える
    has_child = np.in1d(ids, parent_unique, assume_unique=True)

    # ルートか
    is_root = pd.isna(parents)

    # 三分岐はブール代入で埋める（np.where ネストよりわずかに軽い）
    out_type = np.empty(ids.shape[0], dtype=object)
    out_type[is_root] = 'Root'
    non_root = ~is_root
    out_type[non_root & has_child] = 'Inner'
    out_type[non_root & ~has_child] = 'Leaf'

    # 出力（列順固定、並べ替え無し）
    return pd.DataFrame({'id': ids, 'type': out_type})

Analyze Complexity
Runtime 263 ms
Beats 99.37%
Memory 66.91 MB
Beats 73.85%

```

### マイクロ最適化の意図

* **Index/Series を作らない**: `pd.Index(...)` や `Series.isin` は便利ですがオーバーヘッド増。
  → `pd.unique` + `np.in1d` に寄せると軽くなりやすいです。
* **`assume_unique=True`**: `id` は一意、`parent_unique` はすでにユニーク → `in1d` が高速化。
* **三分岐は代入**: `np.where` ネストより、事前に `non_root` を作って3本のブール代入で埋める方が薄い計算量で済みます。
* **コピー削減**: 中間 DataFrame を作らず、配列で完結。

## 3) アルゴリズム説明

* 使用 API

  * `Series.to_numpy()`：配列化で軽量計算
  * `pd.unique`：`p_id` のユニーク親集合（NaN 除去後）を抽出
  * `np.in1d(..., assume_unique=True)`：`id` が親集合に含まれるかを高速判定
* **NULL / 重複 / 型**

  * ルート判定は `pd.isna(p_s)` を明示
  * `id` は仕様上一意 → `assume_unique=True` を安全に使用
  * 返却 dtype は `type=object`（審査系の比較に無難）。
    もし許容されるなら `astype('category')` でメモリ削減可（下記参照）

## 4) 計算量（概算）

* 親集合作成：`O(N)`
* 包含判定（`in1d`）：`O(N + U)`（U は `parent_unique` のサイズ）
* 全体：**O(N)**、追加メモリは `parent_unique` と `out_type` 分のみ

## 追加の微調整（必要に応じて）

* **メモリ削減（返却時のみ）**
  審査が dtype 非依存なら:

  ```python
  # 返却直前
  # df = pd.DataFrame({'id': ids, 'type': out_type})
  # df['type'] = pd.Categorical(df['type'], categories=['Root','Inner','Leaf'])
  ```

  これで `type` 列を category 化し、数十 % メモリを圧縮できます（Runtime 影響は小〜微増）。

* **nullable 整数のまま扱いたい場合**
  入力が `Int64`（nullable 整数）なら、そのまま配列化しても問題ありません。
  内部的には `NaN` 判定のみ行い、`id` は値をそのまま返します。

---

**まとめ**
現状の 282ms/67MB は十分優秀ですが、上記の**配列直叩き版**は多くの環境でさらに軽くなる傾向があります。
特に `pd.Index`・`Series.isin` を外し、`pd.unique` + `np.in1d(assume_unique=True)` に寄せるのが効きどころです。

